## Loading Libraries and establishing connection to the database

In [1]:
import pymysql
import pandas as pd
import yfinance as yf
import numpy as np
from nsepy import get_history
from dateutil.relativedelta import relativedelta, TH
from datetime import date, timedelta
from nsepy.derivatives import get_expiry_date

# conn = pymysql.connect(host = '130.211.219.71',port=int(3306),user='root',passwd='Nodal@987',db='nodalDB')
# cur = conn.cursor()

## Nifty and Banknifty stocks

In [2]:
nifty = pd.read_csv("nifty.csv")
banknifty = pd.read_csv("banknifty.csv")

In [3]:
nifty_list = nifty['Symbol'].tolist()
banknifty_list = banknifty['Symbol'].tolist()

In [4]:
print(nifty_list)
print(len(nifty_list))
print(banknifty_list)
print(len(banknifty_list))
final_list = list(sorted(set(nifty_list) | set(banknifty_list)))
print(final_list)
print(len(final_list))

['ADANIPORTS', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'HDFC', 'ICICIBANK', 'ITC', 'IOC', 'INDUSINDBK', 'INFY', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NTPC', 'NESTLEIND', 'ONGC', 'POWERGRID', 'RELIANCE', 'SBILIFE', 'SHREECEM', 'SBIN', 'SUNPHARMA', 'TCS', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'TITAN', 'UPL', 'ULTRACEMCO', 'WIPRO']
50
['AUBANK', 'AXISBANK', 'BANDHANBNK', 'FEDERALBNK', 'HDFCBANK', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'KOTAKBANK', 'PNB', 'RBLBANK', 'SBIN']
12
['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO'

## Function to add data to the database table

In [19]:
def get_thurs(dt):
    return (dt + relativedelta(day=31, weekday=TH(-1)))

get_thurs(date(2021,11,4))

datetime.date(2021, 11, 25)

In [20]:
# define and month year range to loop over
month_list = np.arange(1,2, step = 1); print(month_list)
yr_list = np.arange(2015, 2016, step = 1 ); print(yr_list)

[1]
[2015]


In [21]:
# empty dataframe
TATA_data = pd.DataFrame() # to use in the loop
TATA_option_data = pd.DataFrame() # to store output
symbol = "TECHM"

In [22]:
# break the loop into 2 parts to avoid having to rerun everything if there are querying errors
for yr in yr_list:
    # loop through all the months and years
    counter = 0
    print('Year: ', yr)
    for mnth in month_list:
        current_dt = date(yr, mnth, 1)
        start_dt = current_dt + relativedelta(days = -4)
#         end_dt = list(get_expiry_date(year = yr, month = mnth))
        last_thur = get_thurs(date(yr,mnth,1))
        e_data = last_thur
#         print(e_data)
#         print(start_dt)
#         end_dt.sort()
        # print('current: ', current_dt)
        # print('start: ', start_dt)
        # print('end: ', end_dt)
        print('Months: ', mnth)
        # print(month) 
        # get TATA nifty futures data
        
        TATA_fut = get_history(symbol = symbol,
                               start = start_dt, 
                               end = e_data)
        print(TATA_fut)
        TATA_data = TATA_data.append(TATA_fut)

        # calculate high and low values for each month; round off to get strike prices
        high = TATA_fut['Close'].max()
        high = int(round(high/1)*1) # ; print('High:', high)
#         print("High is : ",high)
        low = TATA_fut['Close'].min()
        low = int(round(low/1)*1) # ; print('Low :', low)
#         print("Low is :",low)

        for strike in range(low, high, 1): # start, stop, step
            """
            get daily closing nifty index option prices for 3 months 
            over the entire range 
            """
            # time.sleep(random.randint(10,20)) # pause for random interval so as to not overwhelm the site
            TATA_opt = get_history(symbol = symbol,
                                   start = start_dt, end = e_data,
                                   option_type = 'PE',
                                   strike_price = strike,
                                   expiry_date = e_data)

            if e_data == last_thur:
                TATA_opt['expiry_type'] = ["monthly"] * TATA_opt.shape[0]
            else:
                TATA_opt['expiry_type'] = ["weekly"] * TATA_opt.shape[0]

            TATA_option_data = TATA_option_data.append(TATA_opt)

            #time.sleep(random.randint(20,50)) # pause for random interval so as to not overwhelm the site
            TATA_opt = get_history(symbol = symbol,
                                   start = start_dt, end = e_data,
                                   option_type = 'CE',
                                   strike_price = strike,
                                   expiry_date = e_data)

            if e_data == last_thur:
                TATA_opt['expiry_type'] = ["monthly"] * TATA_opt.shape[0]
            else:
                TATA_opt['expiry_type'] = ["weekly"] * TATA_opt.shape[0]
            TATA_option_data = TATA_option_data.append(TATA_opt)
        
#         TATA_option_data.reset_index(inplace = True)
#         df = pd.DataFrame(columns = TATA_option_data.columns)
#         df2 = pd.DataFrame(columns = TATA_option_data.columns)


Year:  2015
Months:  1
           Symbol Series  Prev Close     Open     High      Low     Last  \
Date                                                                       
2014-12-29  TECHM     EQ     2566.70  2562.00  2594.00  2562.00  2592.70   
2014-12-30  TECHM     EQ     2589.75  2590.40  2630.40  2590.00  2611.00   
2014-12-31  TECHM     EQ     2611.65  2620.00  2622.85  2585.00  2593.30   
2015-01-01  TECHM     EQ     2593.55  2589.00  2613.45  2580.75  2596.00   
2015-01-02  TECHM     EQ     2594.55  2587.00  2619.00  2587.00  2611.00   
2015-01-05  TECHM     EQ     2607.25  2600.00  2619.70  2555.00  2559.20   
2015-01-06  TECHM     EQ     2563.75  2555.05  2569.00  2515.35  2541.00   
2015-01-07  TECHM     EQ     2536.25  2545.00  2574.80  2513.20  2534.00   
2015-01-08  TECHM     EQ     2524.40  2528.00  2579.90  2528.00  2569.30   
2015-01-09  TECHM     EQ     2566.65  2568.00  2689.90  2568.00  2683.10   
2015-01-12  TECHM     EQ     2680.35  2697.00  2759.00  2685.00  

In [23]:
g = TATA_option_data.copy()
g.head(100)

,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying,expiry_type
Date,,,,,,,,,,,,,,,,,
2014-12-29,TECHM,2015-01-29,PE,2550.0,53.70,57.00,47.50,48.30,47.85,48.30,106,34482000.0,NaN,17375,2750,2589.75,monthly
2014-12-30,TECHM,2015-01-29,PE,2550.0,47.40,50.80,39.00,45.15,44.20,45.15,94,30476000.0,NaN,21500,4125,2611.65,monthly
2014-12-31,TECHM,2015-01-29,PE,2550.0,43.10,51.00,43.10,48.45,48.15,48.45,89,28896000.0,NaN,22000,500,2593.55,monthly
2015-01-01,TECHM,2015-01-29,PE,2550.0,48.10,50.45,43.95,47.20,47.00,47.20,54,17539000.0,NaN,20875,-1125,2594.55,monthly
2015-01-02,TECHM,2015-01-29,PE,2550.0,41.25,43.85,37.70,42.00,41.90,42.00,27,8744000.0,NaN,20750,-125,2607.25,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-01,TECHM,2015-01-29,PE,2650.0,93.10,100.30,87.10,93.15,93.95,93.15,11,3774000.0,NaN,4250,-500,2594.55,monthly
2015-01-02,TECHM,2015-01-29,PE,2650.0,82.60,90.40,77.75,86.75,80.85,86.75,57,19482000.0,NaN,4500,250,2607.25,monthly
2015-01-05,TECHM,2015-01-29,PE,2650.0,79.00,103.95,79.00,103.65,103.65,103.65,9,3086000.0,NaN,4750,250,2563.75,monthly


In [24]:
col = g.columns
col

Index(['Symbol', 'Expiry', 'Option Type', 'Strike Price', 'Open', 'High',
       'Low', 'Close', 'Last', 'Settle Price', 'Number of Contracts',
       'Turnover', 'Premium Turnover', 'Open Interest', 'Change in OI',
       'Underlying', 'expiry_type'],
      dtype='object')

In [25]:
# empty dataframe
TATA_data = pd.DataFrame() # to use in the loop
TATA_option_data = pd.DataFrame() # to store output
df = pd.DataFrame(columns = col)
df2 = pd.DataFrame(columns = col)
month_list = np.arange(1,13 , step = 1); print(month_list)
yr_list = np.arange(2015, 2022, step = 1 ); print(yr_list)

[ 1  2  3  4  5  6  7  8  9 10 11 12]
[2015 2016 2017 2018 2019 2020 2021]


In [26]:
df

,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying,expiry_type


In [28]:
# break the loop into 2 parts to avoid having to rerun everything if there are querying errors
for yr in yr_list:
    # loop through all the months and years
    counter = 0
    print('Year: ', yr)
    for mnth in month_list:
        monthly = pd.DataFrame() # to store output
        underlying = pd.DataFrame()
        current_dt = date(yr, mnth, 1)
#         start_dt = current_dt + relativedelta(days = -4)
        start_dt = current_dt
#         end_dt = list(get_expiry_date(year = yr, month = mnth))
        last_thur = get_thurs(date(yr,mnth,1))
        e_data = last_thur
        print(e_data)
        print(start_dt)
#         end_dt.sort()
        # print('current: ', current_dt)
        # print('start: ', start_dt)
        # print('end: ', end_dt)
        print('Months: ', mnth)
        # print(month) 
        # get TATA nifty futures data
        
        TATA_fut = get_history(symbol = symbol,
                               start = start_dt, 
                               end = e_data)
#         print(TATA_fut)
        TATA_data = TATA_data.append(TATA_fut)
        underlying = underlying.append(TATA_fut)
        # calculate high and low values for each month; round off to get strike prices
        high = TATA_fut['Close'].max()
        high = int(round(high/1)*1) # ; print('High:', high)
        print("High is : ",high)
        low = TATA_fut['Close'].min()
        low = int(round(low/1)*1) # ; print('Low :', low)
        print("Low is :",low)

        for strike in range(low, high + 1, 1): # start, stop, step
            """
            get daily closing nifty index option prices for 3 months 
            over the entire range 
            """
            # time.sleep(random.randint(10,20)) # pause for random interval so as to not overwhelm the site
            TATA_opt = get_history(symbol = symbol,
                                   start = start_dt, end = e_data,
                                   option_type = 'PE',
                                   strike_price = strike,
                                   expiry_date = e_data)

            if e_data == last_thur:
                TATA_opt['expiry_type'] = ["monthly"] * TATA_opt.shape[0]
            else:
                TATA_opt['expiry_type'] = ["weekly"] * TATA_opt.shape[0]
            
            monthly = monthly.append(TATA_opt)
            TATA_option_data = TATA_option_data.append(TATA_opt)

            #time.sleep(random.randint(20,50)) # pause for random interval so as to not overwhelm the site
            TATA_opt = get_history(symbol = symbol,
                                   start = start_dt, end = e_data,
                                   option_type = 'CE',
                                   strike_price = strike,
                                   expiry_date = e_data)

            if e_data == last_thur:
                TATA_opt['expiry_type'] = ["monthly"] * TATA_opt.shape[0]
            else:
                TATA_opt['expiry_type'] = ["weekly"] * TATA_opt.shape[0]
            
            monthly = monthly.append(TATA_opt)
            TATA_option_data = TATA_option_data.append(TATA_opt)
        monthly['value'] = underlying['Close']   
        monthly.reset_index(inplace = True)
        t = list(monthly['Strike Price'].unique())
        print(t)
        for i in range(monthly.shape[0]):
            distance = []
            for j in monthly['Strike Price'].unique():
                distance.append(abs(monthly['value'][i].tolist() - j))
            min_value = min(distance)
            min_index = distance.index(min_value)
            y = monthly[monthly['Strike Price'] == t[min_index]]
            df2 = y[y['Date'] == monthly['Date'][i]]
            df = df.append(df2, ignore_index=True)

Year:  2015
2015-01-29
2015-01-01
Months:  1
High is :  2904
Low is : 2524


KeyboardInterrupt: 

In [ ]:
# TATA_opt = get_history(symbol = 'ASIANPAINT',
#                                    start = date(2020,1,1), end = date(2020,1,30),
#                                    option_type = 'CE',
#                                    strike_price = 1850,
#                                    expiry_date = date(2020,1,30))
# TATA_opt.head()

In [ ]:
# TATA_fut = get_history(symbol = 'ADANIPORTS',
#                                start = date(2015,1,1), 
#                                end = date(2015,1,29))
# TATA_fut.head(30)

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.head(100)

In [ ]:
df.to_csv("stock options/" + str(symbol) + ".csv")

In [110]:
# TATA_option_data.reset_index(inplace = True)
# TATA_option_data.head(100)

,Date,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying,expiry_type
0,2014-12-29,TATAMOTORS,2015-01-29,PE,490.0,13.95,13.95,10.25,11.20,11.35,11.20,576,144444000.0,NaN,203000,38000,495.05,monthly
1,2014-12-30,TATAMOTORS,2015-01-29,PE,490.0,9.55,13.30,9.55,11.75,11.40,11.75,334,83804000.0,NaN,232000,29000,493.75,monthly
2,2014-12-31,TATAMOTORS,2015-01-29,PE,490.0,12.00,14.00,9.50,10.45,10.50,10.45,874,219025000.0,NaN,302000,70000,495.55,monthly
3,2015-01-01,TATAMOTORS,2015-01-29,PE,490.0,10.95,11.65,9.15,9.45,9.40,9.45,226,56532000.0,NaN,294500,-7500,498.45,monthly
4,2015-01-02,TATAMOTORS,2015-01-29,PE,490.0,8.45,8.45,4.65,5.60,5.35,5.60,791,196172000.0,NaN,247000,-47500,511.80,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2015-01-01,TATAMOTORS,2015-01-29,PE,510.0,20.60,20.60,19.10,19.45,19.30,19.45,36,9531000.0,NaN,60000,13500,498.45,monthly
96,2015-01-02,TATAMOTORS,2015-01-29,PE,510.0,17.00,17.00,10.30,12.60,11.65,12.60,759,198261000.0,NaN,115500,55500,511.80,monthly
97,2015-01-05,TATAMOTORS,2015-01-29,PE,510.0,10.05,14.10,7.45,7.80,8.00,7.80,1023,266024000.0,NaN,140000,24500,523.80,monthly
98,2015-01-06,TATAMOTORS,2015-01-29,PE,510.0,11.15,19.50,11.00,17.85,17.90,17.85,1295,340486000.0,NaN,123500,-16500,501.40,monthly


In [111]:
t = list(TATA_option_data['Strike Price'].unique())
t

[490.0, 500.0, 510.0, 520.0, 530.0, 540.0, 550.0, 560.0, 570.0, 580.0, 590.0]

In [112]:
TATA_option_data['Date'][0]

datetime.date(2014, 12, 29)

In [117]:
df = pd.DataFrame(columns = TATA_option_data.columns)
df2 = pd.DataFrame(columns = TATA_option_data.columns)

In [120]:

for i in range(TATA_option_data.shape[0]):
    distance = []
    for j in TATA_option_data['Strike Price'].unique():
        distance.append(abs(TATA_option_data['Underlying'][i].tolist() - j))
    min_value = min(distance)
    min_index = distance.index(min_value)
    y = TATA_option_data[TATA_option_data['Strike Price'] == t[min_index]]
    df2 = y[y['Date'] == TATA_option_data['Date'][i]]
    df = df.append(df2, ignore_index=True)

          Date      Symbol      Expiry Option Type  Strike Price  Open  High  \
46  2014-12-29  TATAMOTORS  2015-01-29          PE         500.0  17.8  17.9   
69  2014-12-29  TATAMOTORS  2015-01-29          CE         500.0  11.3  16.3   

     Low  Close   Last  Settle Price Number of Contracts     Turnover  \
46  14.4  15.55  16.00         15.55                 472  121700000.0   
69  11.3  15.00  14.85         15.00                1970  507124000.0   

   Premium Turnover Open Interest Change in OI  Underlying expiry_type  
46              NaN        162500        32500      495.05     monthly  
69              NaN        590000        84000      495.05     monthly  
          Date      Symbol      Expiry Option Type  Strike Price   Open  High  \
1   2014-12-30  TATAMOTORS  2015-01-29          PE         490.0   9.55  13.3   
24  2014-12-30  TATAMOTORS  2015-01-29          CE         490.0  21.30  21.7   

      Low  Close  Last  Settle Price Number of Contracts    Turnover  \
1   

           Date      Symbol      Expiry Option Type  Strike Price   Open  \
150  2015-01-14  TATAMOTORS  2015-01-29          PE         520.0  10.25   
173  2015-01-14  TATAMOTORS  2015-01-29          CE         520.0  15.45   

     High    Low  Close   Last  Settle Price Number of Contracts     Turnover  \
150  16.0   8.45   11.7  11.55          11.7                1549  412579000.0   
173  16.8  10.45   13.8  14.05          13.8                1734  462248000.0   

    Premium Turnover Open Interest Change in OI  Underlying expiry_type  
150              NaN        323000        40000      520.35     monthly  
173              NaN        714000        47000      520.35     monthly  
           Date      Symbol      Expiry Option Type  Strike Price   Open  \
197  2015-01-15  TATAMOTORS  2015-01-29          PE         530.0  11.20   
220  2015-01-15  TATAMOTORS  2015-01-29          CE         530.0  11.85   

     High    Low  Close   Last  Settle Price Number of Contracts     Turnove

505              NaN        160500      -147000       598.6     monthly  
          Date      Symbol      Expiry Option Type  Strike Price  Open  High  \
46  2014-12-29  TATAMOTORS  2015-01-29          PE         500.0  17.8  17.9   
69  2014-12-29  TATAMOTORS  2015-01-29          CE         500.0  11.3  16.3   

     Low  Close   Last  Settle Price Number of Contracts     Turnover  \
46  14.4  15.55  16.00         15.55                 472  121700000.0   
69  11.3  15.00  14.85         15.00                1970  507124000.0   

   Premium Turnover Open Interest Change in OI  Underlying expiry_type  
46              NaN        162500        32500      495.05     monthly  
69              NaN        590000        84000      495.05     monthly  
          Date      Symbol      Expiry Option Type  Strike Price   Open  High  \
1   2014-12-30  TATAMOTORS  2015-01-29          PE         490.0   9.55  13.3   
24  2014-12-30  TATAMOTORS  2015-01-29          CE         490.0  21.30  21.7   

  

           Date      Symbol      Expiry Option Type  Strike Price   Open  \
150  2015-01-14  TATAMOTORS  2015-01-29          PE         520.0  10.25   
173  2015-01-14  TATAMOTORS  2015-01-29          CE         520.0  15.45   

     High    Low  Close   Last  Settle Price Number of Contracts     Turnover  \
150  16.0   8.45   11.7  11.55          11.7                1549  412579000.0   
173  16.8  10.45   13.8  14.05          13.8                1734  462248000.0   

    Premium Turnover Open Interest Change in OI  Underlying expiry_type  
150              NaN        323000        40000      520.35     monthly  
173              NaN        714000        47000      520.35     monthly  
           Date      Symbol      Expiry Option Type  Strike Price   Open  \
197  2015-01-15  TATAMOTORS  2015-01-29          PE         530.0  11.20   
220  2015-01-15  TATAMOTORS  2015-01-29          CE         530.0  11.85   

     High    Low  Close   Last  Settle Price Number of Contracts     Turnove

          Date      Symbol      Expiry Option Type  Strike Price   Open  High  \
1   2014-12-30  TATAMOTORS  2015-01-29          PE         490.0   9.55  13.3   
24  2014-12-30  TATAMOTORS  2015-01-29          CE         490.0  21.30  21.7   

      Low  Close  Last  Settle Price Number of Contracts    Turnover  \
1    9.55  11.75  11.4         11.75                 334  83804000.0   
24  17.30  19.00  19.1         19.00                 102  25947000.0   

   Premium Turnover Open Interest Change in OI  Underlying expiry_type  
1               NaN        232000        29000      493.75     monthly  
24              NaN        116000         5000      493.75     monthly  
          Date      Symbol      Expiry Option Type  Strike Price  Open  High  \
48  2014-12-31  TATAMOTORS  2015-01-29          PE         500.0  14.9  17.2   
71  2014-12-31  TATAMOTORS  2015-01-29          CE         500.0  12.8  15.9   

      Low  Close   Last  Settle Price Number of Contracts     Turnover  \
48  1

...

           Date      Symbol      Expiry Option Type  Strike Price  Open  \
100  2015-01-08  TATAMOTORS  2015-01-29          PE         510.0  16.4   
123  2015-01-08  TATAMOTORS  2015-01-29          CE         510.0  14.8   

      High    Low  Close   Last  Settle Price Number of Contracts  \
100  17.00  10.25   11.6  12.90          11.6                 658   
123  16.45  11.35   14.6  13.75          14.6                1468   

        Turnover Premium Turnover Open Interest Change in OI  Underlying  \
100  171955000.0              NaN        143000        33000       512.0   
123  384483000.0              NaN        294500       -65500       512.0   

    expiry_type  
100     monthly  
123     monthly  
           Date      Symbol      Expiry Option Type  Strike Price   Open  \
147  2015-01-09  TATAMOTORS  2015-01-29          PE         520.0  12.25   
170  2015-01-09  TATAMOTORS  2015-01-29          CE         520.0  12.35   

     High    Low  Close  Last  Settle Price Number of 

           Date      Symbol      Expiry Option Type  Strike Price   Open  \
245  2015-01-19  TATAMOTORS  2015-01-29          PE         540.0  18.50   
268  2015-01-19  TATAMOTORS  2015-01-29          CE         540.0   7.15   

      High   Low  Close  Last  Settle Price Number of Contracts     Turnover  \
245  19.25  11.0  11.70  12.2         11.70                 249   68812000.0   
268  10.50   5.6  10.25   9.9         10.25                2796  767567000.0   

    Premium Turnover Open Interest Change in OI  Underlying expiry_type  
245              NaN        127000        37500       538.5     monthly  
268              NaN        664500       184500       538.5     monthly  
           Date      Symbol      Expiry Option Type  Strike Price  Open  High  \
338  2015-01-20  TATAMOTORS  2015-01-29          PE         560.0  22.3  22.3   
361  2015-01-20  TATAMOTORS  2015-01-29          CE         560.0   3.8  11.0   

       Low  Close   Last  Settle Price Number of Contracts     T

          Date      Symbol      Expiry Option Type  Strike Price   Open  High  \
49  2015-01-01  TATAMOTORS  2015-01-29          PE         500.0  15.60  15.7   
72  2015-01-01  TATAMOTORS  2015-01-29          CE         500.0  13.05  15.2   

      Low  Close   Last  Settle Price Number of Contracts     Turnover  \
49  13.15  13.65  13.25         13.65                 136   34983000.0   
72  13.05  14.90  14.95         14.90                 480  123447000.0   

   Premium Turnover Open Interest Change in OI  Underlying expiry_type  
49              NaN        241000         6500      498.45     monthly  
72              NaN        647500        14000      498.45     monthly  
           Date      Symbol      Expiry Option Type  Strike Price   Open  \
96   2015-01-02  TATAMOTORS  2015-01-29          PE         510.0  17.00   
119  2015-01-02  TATAMOTORS  2015-01-29          CE         510.0  11.05   

     High    Low  Close   Last  Settle Price Number of Contracts     Turnover  \
96  

           Date      Symbol      Expiry Option Type  Strike Price  Open  High  \
338  2015-01-20  TATAMOTORS  2015-01-29          PE         560.0  22.3  22.3   
361  2015-01-20  TATAMOTORS  2015-01-29          CE         560.0   3.8  11.0   

       Low  Close   Last  Settle Price Number of Contracts     Turnover  \
338  10.70   11.4  11.25          11.4                 246   70411000.0   
361   3.05   10.1  10.60          10.1                2743  777649000.0   

    Premium Turnover Open Interest Change in OI  Underlying expiry_type  
338              NaN         71000        52500       559.2     monthly  
361              NaN        381000        15000       559.2     monthly  
           Date      Symbol      Expiry Option Type  Strike Price  Open  High  \
293  2015-01-21  TATAMOTORS  2015-01-29          PE         550.0   7.8  13.0   
316  2015-01-21  TATAMOTORS  2015-01-29          CE         550.0  12.4  15.4   

      Low  Close   Last  Settle Price Number of Contracts     Tu

           Date      Symbol      Expiry Option Type  Strike Price   Open  \
96   2015-01-02  TATAMOTORS  2015-01-29          PE         510.0  17.00   
119  2015-01-02  TATAMOTORS  2015-01-29          CE         510.0  11.05   

     High    Low  Close   Last  Settle Price Number of Contracts     Turnover  \
96   17.0  10.30  12.60  11.65         12.60                 759  198261000.0   
119  19.9  11.05  16.95  17.40         16.95                1525  400582000.0   

    Premium Turnover Open Interest Change in OI  Underlying expiry_type  
96               NaN        115500        55500       511.8     monthly  
119              NaN        299000        45500       511.8     monthly  
           Date      Symbol      Expiry Option Type  Strike Price   Open  \
143  2015-01-05  TATAMOTORS  2015-01-29          PE         520.0  15.55   
166  2015-01-05  TATAMOTORS  2015-01-29          CE         520.0  12.95   

      High    Low  Close  Last  Settle Price Number of Contracts     Turnove

In [121]:
df.head()

,Date,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying,expiry_type
0,2014-12-29,TATAMOTORS,2015-01-29,PE,500.0,17.80,17.9,14.40,15.55,16.00,15.55,472,121700000.0,NaN,162500,32500,495.05,monthly
1,2014-12-29,TATAMOTORS,2015-01-29,CE,500.0,11.30,16.3,11.30,15.00,14.85,15.00,1970,507124000.0,NaN,590000,84000,495.05,monthly
2,2014-12-30,TATAMOTORS,2015-01-29,PE,490.0,9.55,13.3,9.55,11.75,11.40,11.75,334,83804000.0,NaN,232000,29000,493.75,monthly
3,2014-12-30,TATAMOTORS,2015-01-29,CE,490.0,21.30,21.7,17.30,19.00,19.10,19.00,102,25947000.0,NaN,116000,5000,493.75,monthly
4,2014-12-31,TATAMOTORS,2015-01-29,PE,500.0,14.90,17.2,12.00,14.75,14.95,14.75,755,194521000.0,NaN,234500,72500,495.55,monthly


In [122]:
df.to_csv("ok.csv")

In [45]:
TATA_option_data.to_csv("TATAMOTORS-dec-2021.csv")

In [31]:
data = get_history(symbol='TATAMOTORS',
                   start=date(2015,1,1),
                   end=date(2015,1,29))
data.head()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2015-01-01,TATAMOTORS,EQ,495.55,494.50,499.45,492.25,498.55,498.45,496.76,934528,4.642335e+13,12769,303443,0.3247
2015-01-02,TATAMOTORS,EQ,498.45,497.25,517.40,497.25,513.50,511.80,510.18,4148596,2.116515e+14,60297,2098181,0.5058
2015-01-05,TATAMOTORS,EQ,511.80,510.10,526.00,506.70,524.10,523.80,518.60,6686952,3.467850e+14,89227,3696954,0.5529
2015-01-06,TATAMOTORS,EQ,523.80,514.35,514.35,499.25,502.00,501.40,504.78,7894918,3.985170e+14,160548,4736934,0.6000
2015-01-07,TATAMOTORS,EQ,501.40,495.00,500.40,489.20,493.70,493.85,494.52,9134678,4.517269e+14,166649,5515549,0.6038


In [36]:
stock_opt = get_history(symbol="TATAMOTORS",
                        start=date(2015,1,1),
                        end=date(2015,1,29),
                        option_type="CE",
                        strike_price=490,
                        expiry_date=date(2015,1,29))

In [37]:
stock_opt.head(25)
# stock_opt.info()

,Symbol,Expiry,Option Type,Strike Price,Open,High,Low,Close,Last,Settle Price,Number of Contracts,Turnover,Premium Turnover,Open Interest,Change in OI,Underlying
Date,,,,,,,,,,,,,,,,
2015-01-01,TATAMOTORS,2015-01-29,CE,490.0,18.05,20.90,18.05,20.85,20.90,20.85,37,9438000.0,NaN,130000,-3000,498.45
2015-01-02,TATAMOTORS,2015-01-29,CE,490.0,21.00,32.20,21.00,29.95,31.00,29.95,131,33890000.0,NaN,102500,-27500,511.80
2015-01-05,TATAMOTORS,2015-01-29,CE,490.0,29.00,40.20,26.20,39.45,39.00,39.45,60,15762000.0,NaN,93500,-9000,523.80
2015-01-06,TATAMOTORS,2015-01-29,CE,490.0,25.35,26.30,21.40,21.75,21.80,21.75,99,25363000.0,NaN,117000,23500,501.40
2015-01-07,TATAMOTORS,2015-01-29,CE,490.0,17.50,22.60,15.80,18.40,18.85,18.40,395,100434000.0,NaN,131500,14500,493.85
2015-01-08,TATAMOTORS,2015-01-29,CE,490.0,22.70,30.00,22.50,27.80,27.10,27.80,94,24247000.0,NaN,123000,-8500,512.00
2015-01-09,TATAMOTORS,2015-01-29,CE,490.0,34.45,38.20,30.80,36.55,35.55,36.55,62,16272000.0,NaN,114000,-9000,522.30
2015-01-12,TATAMOTORS,2015-01-29,CE,490.0,28.25,38.00,28.25,34.85,34.85,34.85,23,6002000.0,NaN,108500,-5500,522.30
2015-01-13,TATAMOTORS,2015-01-29,CE,490.0,31.25,38.05,30.20,33.65,33.65,33.65,22,5770000.0,NaN,104500,-4000,519.45


## Fetching the data from the database table

In [ ]:
df = pd.read_sql_query("SELECT * FROM y_finance_daily ",conn)
df.head()